<a href="https://colab.research.google.com/github/riotrip/ml-smt5/blob/main/TG1_2_3_JS11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Rio Tri Prayogo - 2341720236 - TI 3F/25**
---
## **JS11 - Klasifikasi 2**

### **Tugas Lab**

1. Buatlah model SVM dengan menggunakan data voice.csv dengan ketentuan,

a. Split data dengan rasio 70:30 dan 80:20 untuk setiap model yang akan dibangun.

i. Gunakan model dengan kernel linier.

ii. Gunakan model dengan kernel polynomial.

iii. Gunakan model dengan kernel RBF.

b. Tabulasikan performansi setiap split dan kernel berdasarkan metrik akurasi.

Import Library

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Load Data

In [2]:
data = pd.read_csv('/content/drive/MyDrive/PembelajaranMesin_Rio/docs/voice.csv')
X = data.drop('label', axis=1)
y = data['label'].map({'male': 0, 'female': 1})

Standarisasi

In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Test berbagai split dan kernel


In [4]:
results = []
for split in [0.7, 0.8]:
    for kernel in ['linear', 'poly', 'rbf']:
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=1-split, random_state=42
        )

        model = SVC(kernel=kernel, random_state=42)
        model.fit(X_train, y_train)

        acc = accuracy_score(y_test, model.predict(X_test))
        results.append([f"{int(split*100)}:{int((1-split)*100)}", kernel, round(acc, 4)])

Hasil

In [5]:
print("voice.csv Hasil:")
for split, kernel, acc in results:
    print(f"Split {split}, Kernel {kernel}: {acc}")

best = max(results, key=lambda x: x[2])
print(f"\nBEST: Split {best[0]}, Kernel {best[1]}, Accuracy {best[2]}")

voice.csv Hasil:
Split 70:30, Kernel linear: 0.9706
Split 70:30, Kernel poly: 0.9569
Split 70:30, Kernel rbf: 0.9811
Split 80:19, Kernel linear: 0.9763
Split 80:19, Kernel poly: 0.97
Split 80:19, Kernel rbf: 0.9826

BEST: Split 80:19, Kernel rbf, Accuracy 0.9826


2. Gunakan data pada praktikum 5 untuk membuat model klasifikasi siang dan malam menggunakan SVM dengan kernel RBF menggunakan fitur histrogram. Gunakan rasio 80:20. Anda dapat bereksperimen dengan hyperparameter tunning dari kernel RBF. Catat performansi akurasinya!

Import Library

In [6]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Function ekstrak histogram

In [7]:
def get_histogram(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.resize(img, (100, 100))

    # Histogram RGB
    hist_b = cv2.calcHist([img], [0], None, [32], [0, 256])
    hist_g = cv2.calcHist([img], [1], None, [32], [0, 256])
    hist_r = cv2.calcHist([img], [2], None, [32], [0, 256])

    return np.concatenate([hist_b.flatten(), hist_g.flatten(), hist_r.flatten()])

Function Load Gambar

In [8]:
# Load gambar
def load_images(directory):
    features, labels = [], []

    for label, folder in enumerate(['day', 'night']):
        folder_path = os.path.join(directory, folder)
        if not os.path.exists(folder_path):
            continue

        for file in os.listdir(folder_path):
            if file.endswith(('.jpg', '.png')):
                hist = get_histogram(os.path.join(folder_path, file))
                if hist is not None:
                    features.append(hist)
                    labels.append(label)

    return np.array(features), np.array(labels)

Load data

In [9]:
train_dir = "/content/drive/MyDrive/PembelajaranMesin_Rio/img/images/training/"
test_dir = "/content/drive/MyDrive/PembelajaranMesin_Rio/img/images/test/"

print("Loading images...")
X_train, y_train = load_images(train_dir)
X_test, y_test = load_images(test_dir)

Loading images...


Standardisasi

In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Tuning RBF

In [11]:
print("RBF TUNING:")
best_acc = 0
for C in [1, 10, 100]:
    for gamma in [0.01, 0.1, 'scale']:
        model = SVC(kernel='rbf', C=C, gamma=gamma)
        model.fit(X_train_scaled, y_train)
        acc = accuracy_score(y_test, model.predict(X_test_scaled))
        print(f"C: {C}, gamma: {gamma} -> {acc:.3f}")

        if acc > best_acc:
            best_acc = acc
            best_C, best_gamma = C, gamma

print(f"\nBEST: C={best_C}, gamma={best_gamma}, Accuracy={best_acc:.3f}")

RBF TUNING:
C: 1, gamma: 0.01 -> 0.956
C: 1, gamma: 0.1 -> 0.881
C: 1, gamma: scale -> 0.956
C: 10, gamma: 0.01 -> 0.944
C: 10, gamma: 0.1 -> 0.887
C: 10, gamma: scale -> 0.950
C: 100, gamma: 0.01 -> 0.944
C: 100, gamma: 0.1 -> 0.887
C: 100, gamma: scale -> 0.950

BEST: C=1, gamma=0.01, Accuracy=0.956
